# Configuration

In [15]:
# Import Packages
import os
import cv2
import time
import math
import glob
import torch
import random
import zipfile
import numpy as np
import pandas as pd
import configparser
import matplotlib.pyplot as plt
import torch.nn.functional as Fnn
import torchvision.transforms.functional as TF
#
from pathlib import Path
from datetime import datetime
from torch.utils.data import DataLoader
from torchvision.io import read_image, write_jpeg
from torchvision.utils import draw_bounding_boxes
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
#
try:
    from sklearn.metrics import average_precision_score
    SKL_OK = True
except Exception:
    SKL_OK = False

In [ ]:
# Define constant, global variables
DRIVE_PROJECT_DIR = os.path.abspath(os.getcwd())
ZIP_NAME          = "MOT16.zip"
TEST_SEQ          = "MOT16-03"
RUN_ALL_TEST_SEQ  = False
DATA_SPLIT_RATIO  = 0.8
EPOCHS            = 5
BATCH_SIZE        = 2
CONF_THRESHOLD    = 0.7
IOU_THRESHOLD     = 0.5
SEED              = 42
#
PROJECT_DIR   = f"{DRIVE_PROJECT_DIR}/Faster RCNN"
ZIP_PATH      = f"Faster RCNN/MOT_16-Data/{ZIP_NAME}"
DATA_DIR      = "MOT16"
MODEL_DIR     = f"{PROJECT_DIR}/models_FasterRCNN"
RESULT_DIR    = f"{PROJECT_DIR}/results_FasterRCNN"

In [17]:
# Setup constant variables for the object box detection and tracking settings
FRAME_COL  = 0
ID_COL     = 1
LEFT_COL   = 2
TOP_COL    = 3
WIDTH_COL  = 4
HEIGHT_COL = 5
PERSON_COL = 6

NUM_CLASSES = 2
MODEL_PATH  = f"{MODEL_DIR}/bbox_detector.pth"

In [18]:
# Derive the seed for the project
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# Data Augmentation Classes

In [19]:
# Compose class: Chains a sequence of transformations together.
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

# ToTensor class: (Placeholder) Applies minimal/no transformation; used for interface compatibility.
class ToTensor:
    def __call__(self, image, target):
        return image, target

# RandomHorizontalFlip class: Randomly horizontally flips the image and its bounding boxes with probability p.
class RandomHorizontalFlip:
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, image, target):
        if random.random() < self.p and "boxes" in target and len(target["boxes"]) > 0:
            _, h, w = image.shape
            image = TF.hflip(image)
            boxes = target["boxes"].clone()
            boxes[:, [0, 2]] = w - boxes[:, [2, 0]]
            target["boxes"] = boxes
        return image, target

# RandomScale class: Randomly resizes the image and scales the bounding boxes accordingly within a range.
class RandomScale:
    def __init__(self, scale_range=(0.9, 1.1)):
        self.scale_range = scale_range

    def __call__(self, image, target):
        _, h, w = image.shape
        s = random.uniform(*self.scale_range)
        new_h, new_w = int(h*s), int(w*s)
        image = TF.resize(image, [new_h, new_w])
        if "boxes" in target and len(target["boxes"]) > 0:
            boxes = target["boxes"].clone()
            boxes[:, [0, 2]] *= (new_w / w)
            boxes[:, [1, 3]] *= (new_h / h)
            target["boxes"] = boxes
        return image, target

# ColorJitterLite class: Randomly perturbs image brightness, contrast, and saturation.
class ColorJitterLite:
    def __init__(self, b=0.2, c=0.2, s=0.2):
        self.b, self.c, self.s = b, c, s

    def __call__(self, image, target):
        if random.random() < 0.8:
            image = TF.adjust_brightness(image, random.uniform(1-self.b, 1+self.b))
        if random.random() < 0.8:
            image = TF.adjust_contrast(image, random.uniform(1-self.c, 1+self.c))
        if random.random() < 0.8:
            image = TF.adjust_saturation(image, random.uniform(1-self.s, 1+self.s))
        return image, target

# RandomGaussianBlur class: Applies Gaussian blur to the image with probability p.
class RandomGaussianBlur:
    def __init__(self, p=0.25, kernel_size=3, sigma=(0.1, 1.0)):
        self.p = p
        self.kernel_size = kernel_size
        self.sigma = sigma

    def __call__(self, image, target):
        if random.random() < self.p:
            image = TF.gaussian_blur(image, kernel_size=self.kernel_size, sigma=random.uniform(*self.sigma))
        return image, target

# CutOut class: Randomly masks out square patches from the image with probability p.
class CutOut:
    def __init__(self, n_holes=2, length=24, p=0.25):
        self.n_holes = n_holes
        self.length = length
        self.p = p

    def __call__(self, image, target):
        if random.random() > self.p:
            return image, target
        _, h, w = image.shape
        img = image.clone()
        for _ in range(self.n_holes):
            y = random.randint(0, max(0, h - self.length))
            x = random.randint(0, max(0, w - self.length))
            img[:, y:y+self.length, x:x+self.length] = 0
        return img, target

def get_train_transforms():
    return Compose([
        ToTensor(),
        RandomHorizontalFlip(0.5),
        RandomScale((0.9, 1.1)),
        ColorJitterLite(0.2, 0.2, 0.2),
        RandomGaussianBlur(0.25, 3, (0.1, 1.0)),
        CutOut(2, 24, 0.25)
    ])

def get_test_transforms():
    return Compose([ToTensor()])

# Dataset Handling

In [22]:
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(".")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jam/Documents/git/Deep Learning/deep-learning-final-project/Faster RCNN/Faster RCNN/MOT_16-Data/MOT16.zip'

In [7]:
# MOT16TrainDataset is used for loading and processing the training data for MOT16 sequences
class MOT16TrainDataset(torch.utils.data.Dataset):
    def __init__(self, root, seq_list=None, transforms=None):
        """Initializes the MOT16 training dataset.
        Args:
            root (str): Path to MOT16 root directory.
            seq_list (list, optional): List of sequences to load. Loads all if None.
            transforms (callable, optional): Optional transform to be applied.
        """
        self.root = root
        self.transforms = transforms
        self.imgs = []    # List to hold all image relative paths
        self.targets = [] # List to hold all target dicts (annotations)

        # Get all subdirectories (sequences) in root folder
        all_subdirs = sorted(os.listdir(root))
        # Use provided sequence list, otherwise use all found sequences
        sequence = seq_list if seq_list is not None else all_subdirs

        # Iterate over each sequence (video)
        for subdir in sequence:
            img_dir = os.path.join(root, subdir, "img1")         # Directory containing frames
            gt_path = os.path.join(root, subdir, "gt", "gt.txt") # Annotation file path

            # Skip if sequence does not contain both img1 folder and gt file
            if not (os.path.isdir(img_dir) and os.path.exists(gt_path)): continue

            # Get all .jpg frames and sort for chronological order
            frames = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(".jpg")])
            # Store relative image paths (subdir/img1/frame.jpg)
            self.imgs += [os.path.join(subdir, "img1", f) for f in frames]

            # Load ground truth annotations for the sequence
            gt = np.genfromtxt(gt_path, delimiter=",", dtype=float)
            # Keep only rows marked as "person"
            gt = gt[np.where(gt[:, PERSON_COL] == 1)]

            # Extract bounding box columns
            tops   = gt[:, TOP_COL]
            lefts  = gt[:, LEFT_COL]
            bots   = tops + gt[:, HEIGHT_COL]
            rights = lefts + gt[:, WIDTH_COL]
            boxes  = np.column_stack((lefts, tops, rights, bots))

            # Extract person ID column for each annotation
            person_ids = gt[:, ID_COL]

            # For each frame in this sequence
            for i in range(len(frames)):
                d = {}
                # Select the annotations belonging to this frame (frame numbers are 1-based)
                mask = (gt[:, FRAME_COL].astype(int) == (i + 1))
                req_boxes = boxes[mask, :]
                req_ids   = person_ids[mask]
                # Store relevant data in dict for this frame
                d["boxes"] = torch.tensor(req_boxes, dtype=torch.float32)
                d["labels"] = torch.ones(len(req_boxes), dtype=torch.int64)   # Label 1 for 'person' in detection
                d["person_ids"] = torch.tensor(req_ids, dtype=torch.int64)    # Identity for each instance
                self.targets.append(d)
        print("[INFO] Train dataset length:", len(self.imgs))

    def __len__(self):
        """Return total number of images in the dataset."""
        return len(self.imgs)

    def __getitem__(self, idx):
        """Given an index, loads and returns the (image, target) pair.
        Args:
            idx (int): Index of the image in the dataset.
        Returns:
            (image, target): Image tensor and target dict, both possibly transformed.
        """
        if idx >= len(self.imgs):
            return None, None
        # Get relative image path and build absolute path
        img_rel = self.imgs[idx]
        img_path = os.path.join(self.root, img_rel)
        # Load image and normalize to [0, 1] float tensor
        img = read_image(img_path).float() / 255.0
        # Fetch corresponding annotations
        tgt = self.targets[idx]
        # Deep copy tensors (avoids accidental mutation)
        tgt = {k: (v.clone() if torch.is_tensor(v) else v) for k, v in tgt.items()}
        # Apply transforms if provided
        if self.transforms:
            img, tgt = self.transforms(img, tgt)
        return img, tgt

# MOT16EvalDataset is used for loading and processing the evaluation (validation) data, 
# which is similar to training data but does not include identity information.
class MOT16EvalDataset(torch.utils.data.Dataset):
    def __init__(self, root, seq_list=None, transforms=None):
        """Initializes the MOT16 evaluation/validation dataset.
        Args:
            root (str): Path to MOT16 root directory.
            seq_list (list, optional): List of sequences to load. Loads all if None.
            transforms (callable, optional): Optional transform to be applied.
        """
        self.root = root
        self.transforms = transforms
        self.imgs = []    # List to hold all image relative paths
        self.targets = [] # List to hold all target dicts (annotations)
        
        # Get all subdirectories (sequences) in root folder
        all_subdirs = sorted(os.listdir(root))
        # Use provided sequence list, otherwise use all found sequences
        sequence = seq_list if seq_list is not None else all_subdirs

        # Iterate over each sequence (video)
        for subdir in sequence:
            img_dir = os.path.join(root, subdir, "img1")
            gt_path = os.path.join(root, subdir, "gt", "gt.txt")

            # Skip if sequence does not contain both img1 folder and gt file
            if not (os.path.isdir(img_dir) and os.path.exists(gt_path)):
                continue

            # Get all .jpg frames and sort for chronological order
            frames = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(".jpg")])
            # Store relative image paths (subdir/img1/frame.jpg)
            self.imgs += [os.path.join(subdir, "img1", f) for f in frames]

            # Load ground truth annotations for the sequence
            gt = np.genfromtxt(gt_path, delimiter=",", dtype=float)
            # Keep only rows marked as "person"
            gt = gt[np.where(gt[:, PERSON_COL] == 1)]

            # Extract bounding box columns
            tops   = gt[:, TOP_COL]
            lefts  = gt[:, LEFT_COL]
            bots   = tops + gt[:, HEIGHT_COL]
            rights = lefts + gt[:, WIDTH_COL]
            boxes  = np.column_stack((lefts, tops, rights, bots))

            # For each frame in this sequence
            for i in range(len(frames)):
                d = {}
                # Select the annotations belonging to this frame (frame numbers are 1-based)
                mask = (gt[:, FRAME_COL].astype(int) == (i + 1))
                req_boxes = boxes[mask, :]
                # Store relevant data in dict for this frame
                d["boxes"] = torch.tensor(req_boxes, dtype=torch.float32)
                d["labels"] = torch.ones(len(req_boxes), dtype=torch.int64)  # Label 1 for 'person'
                self.targets.append(d)
        print("[INFO] Eval dataset length:", len(self.imgs))

    def __len__(self):
        """Return total number of images in the dataset."""
        return len(self.imgs)

    def __getitem__(self, idx):
        """Given an index, loads and returns the (image, target) pair.
        Args:
            idx (int): Index of the image in the dataset.
        Returns:
            (image, target): Image tensor and target dict, both possibly transformed.
        """
        if idx >= len(self.imgs):
            return None, None
        # Get relative image path and build absolute path
        img_rel = self.imgs[idx]
        img_path = os.path.join(self.root, img_rel)
        # Load image and normalize to [0, 1] float tensor
        img = read_image(img_path).float() / 255.0
        # Fetch corresponding annotations
        tgt = self.targets[idx]
        # Deep copy tensors (avoids accidental mutation)
        tgt = {k: (v.clone() if torch.is_tensor(v) else v) for k, v in tgt.items()}
        # Apply transforms if provided
        if self.transforms:
            img, tgt = self.transforms(img, tgt)
        return img, tgt

# MOT16TestDataset is used for loading images from the test set, where no ground-truth annotation is available
class MOT16TestDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        """Initializes the MOT16 test dataset (no ground truth available).
        Args:
            root (str): Path to a single test MOT16 sequence directory.
            transforms (callable, optional): Optional transform to be applied.
        """
        self.root = root
        self.transforms = transforms
        # List all images under img1/ subfolder, sorted by filename (sequence order)
        self.imgs = sorted([f for f in os.listdir(os.path.join(root, "img1")) if f.lower().endswith(".jpg")])

    def __len__(self):
        """Return total number of images in the test set."""
        return len(self.imgs)

    def __getitem__(self, idx):
        """Given an index, load and return (image, filename).
        Args:
            idx (int): Index of the image in the sequence.
        Returns:
            (image, filename): Image tensor (possibly transformed), and relative filename.
        """
        img_path = os.path.join(self.root, "img1", self.imgs[idx])
        # Load image and normalize to [0, 1] float tensor
        img = read_image(img_path).float() / 255.0
        # Apply transforms if provided (target is dummy empty dict)
        if self.transforms:
            img, _ = self.transforms(img, {})
        return img, self.imgs[idx]


# Model Creation

In [8]:
def get_detector_model(weights_path=None):
    """
    Initializes a Faster R-CNN object detector model, optionally loading pretrained weights.
    Args:
        weights_path (str, optional): Path to the model weights file. If provided and exists,
                                      the model will load these weights. Otherwise, uses default weights.
    Returns:
        model: The initialized (and possibly pretrained) Faster R-CNN detection model.
    """
    # Create a Faster R-CNN model with a ResNet50 backbone, using torchvision's default COCO weights.
    model = fasterrcnn_resnet50_fpn(weights="DEFAULT")
    # Determine the number of input features for the classifier head (output of the ROI head).
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # Replace the prediction head to use the required number of classes for our task.
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)
    # If a weights file is provided and exists, load these fine-tuned/trained weights into the model.
    if weights_path and os.path.exists(weights_path):
        state = torch.load(weights_path, map_location="cpu")
        model.load_state_dict(state)
        print(f"[OK] Loaded weights: {weights_path}")
    else:
        # If weights not supplied or file does not exist, warn user (expected on first training run).
        print("[WARN] Training Weights not loaded (Normal for first time training)")
    # Return the initialized (and possibly pretrained) model.
    return model

# Training

In [9]:
def collate_fn(batch):
    # Filter out any samples where the image is None
    batch = [(img,tgt) for img,tgt in batch if img is not None]
    # Unzip batch into two lists: list of images and list of targets; handle empty batch
    return list(zip(*batch)) if batch else ([], [])

def train(model, data_root, seq_list=None, num_epochs=5, batch_size=2, save_path=MODEL_PATH):
    # Select device ("cuda" if available, else "cpu")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to device

    # Freeze backbone parameters so only head layers will be trained
    for p in model.backbone.parameters(): 
        p.requires_grad = False

    # Collect parameters to optimize (those that require gradients)
    params = [p for p in model.parameters() if p.requires_grad]

    # AdamW optimizer with fixed learning rate and weight decay
    optimizer = torch.optim.AdamW(params, lr=5e-4, weight_decay=1e-4)

    # Build training dataset and dataloader with specified transforms and collate_fn
    ds = MOT16TrainDataset(os.path.join(data_root, "train"), seq_list=seq_list, transforms=get_train_transforms())
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=2, pin_memory=True)

    # Compute total steps and warmup for LR schedule
    total_steps = max(1, len(dl) * num_epochs)
    warmup = max(10, int(0.1 * total_steps))
    def lr_lambda(step):
        # Linear warmup, cosine decay for learning rate
        if step < warmup: 
            return float(step) / float(max(1, warmup))
        prog = (step - warmup) / float(max(1, total_steps - warmup))
        import math
        return 0.5 * (1.0 + math.cos(math.pi * prog))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    step = 0
    for epoch in range(num_epochs):
        model.train()   # Set model to training mode
        epoch_loss = 0  # Track total loss per epoch
        t0 = time.time()  # Start timer

        for imgs, tgts in dl:
            # Move images to device
            imgs = [img.to(device) for img in imgs]
            # Move targets to device (for all tensor values in targets)
            tgts = [{k: (v.to(device) if torch.is_tensor(v) else v) for k, v in t.items()} for t in tgts]
            # Forward pass: get loss dict from model
            loss_dict = model(imgs, tgts)
            # Sum all individual loss components to get total loss
            loss = sum(loss_dict.values())
            # Backpropagation and parameter update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Update LR scheduler
            scheduler.step(); step += 1
            # Accumulate loss
            epoch_loss += loss.item()

        # Print epoch summary: average loss and elapsed time
        print(f"[Epoch {epoch+1}/{num_epochs}] loss={epoch_loss/len(dl):.4f}  time={time.time()-t0:.1f}s")
        # Save model after each epoch
        torch.save(model.state_dict(), save_path)
    print(f"[OK] Training Complete → {save_path}")

# Metrics & Evaluation

In [10]:
def box_iou(boxes1, boxes2):
    area1 = (boxes1[:,2]-boxes1[:,0]).clamp(min=0) * (boxes1[:,3]-boxes1[:,1]).clamp(min=0)
    area2 = (boxes2[:,2]-boxes2[:,0]).clamp(min=0) * (boxes2[:,3]-boxes2[:,1]).clamp(min=0)
    lt = torch.max(boxes1[:,None,:2], boxes2[:,:2])
    rb = torch.min(boxes1[:,None,2:], boxes2[:,2:])
    wh = (rb - lt).clamp(min=0)
    inter = wh[:,:,0]*wh[:,:,1]
    union = area1[:,None] + area2 - inter
    return inter / torch.clamp(union, min=1e-6)

def compute_metrics(all_scores, all_tp, total_gt, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    if len(all_scores)==0:
        results={"mAP":0.0,"Total GT boxes":total_gt,"Total detections":0,"True positives":0,"False positives":0,
                 "Precision_at_conf":0.0,"Recall_at_conf":0.0}
        with open(os.path.join(out_dir,"metrics.txt"),"w") as f:
            for k,v in results.items(): f.write(f"{k}: {v}\n")
        return results

    scores_np = np.array(all_scores)
    tps_np    = np.array(all_tp).astype(np.int32)

    order = np.argsort(-scores_np)
    scores_np = scores_np[order]
    tps_np    = tps_np[order]

    cum_tp = np.cumsum(tps_np)
    cum_fp = np.cumsum(1 - tps_np)

    precision = cum_tp / np.maximum(cum_tp + cum_fp, 1)
    recall    = cum_tp / max(total_gt, 1)

    if SKL_OK:
        from sklearn.metrics import average_precision_score
        ap = float(average_precision_score(tps_np, scores_np))
    else:
        ap=0.0
        for r in np.linspace(0,1,11):
            p = precision[recall>=r].max() if np.any(recall>=r) else 0
            ap += p/11.0

    plt.figure(figsize=(6,5))
    plt.plot(recall, precision)
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR (AP={ap:.4f})"); plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir,"precision_recall_curve.png")); plt.close()

    results = {
        "mAP": ap,
        "Total GT boxes": int(total_gt),
        "Total detections": int(len(all_scores)),
        "True positives": int(cum_tp[-1]),
        "False positives": int(cum_fp[-1]),
        "Precision_at_conf": float(precision[-1]),
        "Recall_at_conf": float(recall[-1]),
    }
    with open(os.path.join(out_dir,"metrics.txt"),"w") as f:
        for k,v in results.items(): f.write(f"{k}: {v}\n")
    print("[OK] Writing metrics to:", os.path.join(out_dir,"metrics.txt"))
    return results

def evaluate_on_folder(model, eval_root, seq_list, out_dir, conf_thr=0.7, iou_thr=0.5, device=None): # Added sequence list
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()

    ds = MOT16EvalDataset(eval_root, seq_list=seq_list, transforms=get_test_transforms())

    all_scores=[]; all_tp=[]; total_gt=0
    os.makedirs(out_dir, exist_ok=True)
    vis_dir = os.path.join(out_dir, "visualizations"); os.makedirs(vis_dir, exist_ok=True)

    for idx in range(len(ds)):
        img, tgt = ds[idx]
        gt_boxes = tgt["boxes"]
        total_gt += len(gt_boxes)

        with torch.no_grad():
            pred = model([img.to(device)])[0]
        scores = pred["scores"].detach().cpu()
        boxes  = pred["boxes"].detach().cpu()
        keep = scores >= conf_thr
        scores, boxes = scores[keep], boxes[keep]

        tps = []
        if len(gt_boxes)>0 and len(boxes)>0:
            iou = box_iou(boxes, gt_boxes)
            matched_gt = set()
            for pi in range(len(boxes)):
                gi = torch.argmax(iou[pi]).item()
                max_iou = iou[pi, gi].item()
                if max_iou >= iou_thr and gi not in matched_gt:
                    tps.append(1); matched_gt.add(gi)
                else:
                    tps.append(0)
        elif len(boxes)>0:
            tps = [0]*len(boxes)

        all_scores += scores.tolist()
        all_tp     += tps

        if idx % 50 == 0:
            labels = [f"{s:.2f}" for s in scores.tolist()]
            vis = draw_bounding_boxes((img*255).to(torch.uint8), boxes, labels=labels, width=2)
            write_jpeg(vis, os.path.join(vis_dir, f"det_{idx:06d}.jpg"))

    return compute_metrics(all_scores, all_tp, total_gt, out_dir)

In [11]:
METRICS_TABLE = f"{RESULT_DIR}/metrics_summary.csv"

def save_metrics_row(results:dict, seq_name:str, model_tag:str, conf_thr:float, notes:str=""):
    row = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "seq": seq_name,
        "model": model_tag,
        "conf_threshold": conf_thr,
        "mAP": results.get("mAP"),
        "total_gt": results.get("Total GT boxes"),
        "total_det": results.get("Total detections"),
        "tp": results.get("True positives"),
        "fp": results.get("False positives"),
        "precision_at_conf": results.get("Precision_at_conf"),
        "recall_at_conf": results.get("Recall_at_conf"),
        "notes": notes,
    }
    df = pd.DataFrame([row])
    os.makedirs(os.path.dirname(METRICS_TABLE), exist_ok=True)
    if os.path.exists(METRICS_TABLE):
        df.to_csv(METRICS_TABLE, mode="a", header=False, index=False)
    else:
        df.to_csv(METRICS_TABLE, index=False)
    print(f"[OK] Add metrics to {METRICS_TABLE}")

# Video Creation

In [12]:
def fasterrcnn_detections_to_txt(model, sequence_dir, out_dir, txt_output, threshold=0.7):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()

    ds = MOT16TestDataset(sequence_dir, transforms=get_test_transforms())
    os.makedirs(out_dir, exist_ok=True)

    if len(ds)==0:
        print(f"[ERR] {sequence_dir}/img1 is empty"); return
    print(f"[INFO] Running inference on frames：{len(ds)} frames: {sequence_dir}")

    with open(txt_output, "w") as f_out:
        for frame_idx in range(len(ds)):
            img, name = ds[frame_idx]
            with torch.no_grad():
                pred = model([img.to(device)])[0]

            boxes  = pred["boxes"].detach().cpu()
            scores = pred["scores"].detach().cpu()

            for box, conf in zip(boxes, scores):
                if conf >= threshold:
                    x1, y1, x2, y2 = box
                    w, h = x2-x1, y2-y1
                    track_id=0 # placeholder ids because detection only

                    f_out.write(f"{frame_idx+1},{track_id},{int(x1)},{int(y1)},{int(w)},{int(h)},{conf},-1,-1,-1\n")
    print(f"[OK] Detections saved to {out_dir}")

def get_sequence_info(sequence_dir):
    seqinfo_path = os.path.join(sequence_dir, "seqinfo.ini")
    config = configparser.ConfigParser()
    config.read(seqinfo_path)
    fps = None
    width = None
    height = None
    fps = config.getint("Sequence", "frameRate", fallback=fps)
    width = config.getint("Sequence", "imWidth", fallback=width)
    height = config.getint("Sequence", "imHeight", fallback=height)
    return fps, (width, height)

def visualize_sequence(sequence_dir, results_file, output_video_path):
    # Load results
    results_df = pd.read_csv(results_file, header=None)
    results_df.columns = ["frame","id","x","y","w","h","conf","x3","y3","z3"]

    # Load frames
    image_paths = sorted(glob.glob(os.path.join(sequence_dir, "img1", "*.jpg")))

    # Video writer
    fps, frame_size = get_sequence_info(sequence_dir)
    first_img = cv2.imread(image_paths[0])
    frame_size = (first_img.shape[1], first_img.shape[0])
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

    # Assign random colors for IDs
    max_id = int(results_df["id"].max()) + 1
    np.random.seed(42)  # reproducible colors
    colors = np.random.randint(0, 255, size=(max_id, 3), dtype=np.uint8)

    # Process frames
    for img_path in image_paths:
        frame_num = int(os.path.splitext(os.path.basename(img_path))[0])
        img = cv2.imread(img_path)
        if img is None:
            continue

        frame_data = results_df[results_df["frame"] == frame_num]
        for _, row in frame_data.iterrows():
            track_id = int(row["id"])
            color = tuple(map(int, colors[track_id]))
            x, y, w_box, h_box = int(row["x"]), int(row["y"]), int(row["w"]), int(row["h"])
            cv2.rectangle(img, (x, y), (x + w_box, y + h_box), color, 2)
            cv2.putText(img, str(track_id), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        video_writer.write(img)
    video_writer.release()
    print(f"Video saved to {output_video_path}")

# MAIN CODE EXECUTION

## Train | Validation | Test Split

In [13]:
def train_eval_split(folders_dir, split_ratio, seed):
    sequences = [d for d in os.listdir(folders_dir) if os.path.isdir(os.path.join(folders_dir, d))]
    sequences.sort()
    print("All sequences:", sequences)

    random.seed(seed)
    random.shuffle(sequences)

    split_idx = int(len(sequences) * split_ratio)
    train_sequence = sequences[:split_idx]
    eval_sequence = sequences[split_idx:]

    print("Training sequences:", train_sequence)
    print("Evaluation sequences:", eval_sequence)

    return train_sequence, eval_sequence

## Train the Model

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_sequence, eval_sequence = train_eval_split(folders_dir=os.path.join(DATA_DIR, "train"), split_ratio=DATA_SPLIT_RATIO, seed=SEED) # Split Data

model = get_detector_model(weights_path=None)
train(model, DATA_DIR, seq_list=train_sequence, num_epochs=EPOCHS, batch_size=BATCH_SIZE, save_path=MODEL_PATH)

FileNotFoundError: [Errno 2] No such file or directory: 'MOT16/train'

## Evaluation

In [ ]:
model = get_detector_model(weights_path=MODEL_PATH)

eval_root = os.path.join(DATA_DIR, "train")
eval_out  = os.path.join(RESULT_DIR, f"eval_train_conf{CONF_THRESHOLD}")
res = evaluate_on_folder(model, eval_root, seq_list=eval_sequence, out_dir=eval_out, conf_thr=CONF_THRESHOLD, iou_thr=IOU_THRESHOLD, device=device)
save_metrics_row(res, seq_name="TRAIN_ALL", model_tag="fasterrcnn_head_finetune", conf_thr=CONF_THRESHOLD, notes="aug+AdamW+warmup_cos")

## Test Video Creation to Visualize Results

In [ ]:
model = get_detector_model(weights_path=MODEL_PATH)

# e) Single sequence or complete sequence (testing = no ground truth)
test_seqs = ["MOT16-01","MOT16-03","MOT16-06","MOT16-07","MOT16-08","MOT16-12","MOT16-14"]
if RUN_ALL_TEST_SEQ:
    for seq in test_seqs:
        sequence_dir = os.path.join(DATA_DIR, "test", seq)
        out_dir = os.path.join(RESULT_DIR, f"test_{seq}")
        text_output = os.path.join(out_dir, f"test_{seq}.txt")
        fasterrcnn_detections_to_txt(model, sequence_dir, out_dir, text_output, threshold=CONF_THRESHOLD) # create txt with detections
        video_path = os.path.join(out_dir, f"test_{seq}.mp4")
        visualize_sequence(sequence_dir, results_file=text_output, output_video_path=video_path) # save video with detections
else:
    seq = TEST_SEQ
    sequence_dir = os.path.join(DATA_DIR, "test", seq)
    out_dir = os.path.join(RESULT_DIR, f"test_{seq}")
    text_output = os.path.join(out_dir, f"test_{seq}.txt")
    fasterrcnn_detections_to_txt(model, sequence_dir, out_dir, text_output, threshold=CONF_THRESHOLD) # create txt with detections
    video_path = os.path.join(out_dir, f"test_{seq}.mp4")
    visualize_sequence(sequence_dir, results_file=text_output, output_video_path=video_path) # save video with detections

print("\n[DONE]")

# Training Video Creation Based on Model

In [ ]:
model = get_detector_model(weights_path=MODEL_PATH)

# e) Single sequence or complete sequence (testing = no ground truth)
test_seqs = ["MOT16-01","MOT16-03","MOT16-06","MOT16-07","MOT16-08","MOT16-12","MOT16-14"]
if RUN_ALL_TEST_SEQ:
    for seq in test_seqs:
        sequence_dir = os.path.join(DATA_DIR, "test", seq)
        out_dir = os.path.join(RESULT_DIR, f"test_{seq}")
        text_output = os.path.join(out_dir, f"test_{seq}.txt")
        fasterrcnn_detections_to_txt(model, sequence_dir, out_dir, text_output, threshold=CONF_THRESHOLD) # create txt with detections
        video_path = os.path.join(out_dir, f"test_{seq}.mp4")
        visualize_sequence(sequence_dir, results_file=text_output, output_video_path=video_path) # save video with detections
else:
    seq = TEST_SEQ
    sequence_dir = os.path.join(DATA_DIR, "test", seq)
    out_dir = os.path.join(RESULT_DIR, f"test_{seq}")
    text_output = os.path.join(out_dir, f"test_{seq}.txt")
    fasterrcnn_detections_to_txt(model, sequence_dir, out_dir, text_output, threshold=CONF_THRESHOLD) # create txt with detections
    video_path = os.path.join(out_dir, f"test_{seq}.mp4")
    visualize_sequence(sequence_dir, results_file=text_output, output_video_path=video_path) # save video with detections

print("\n[DONE]")